<div>
<img src="https://notebooks.dtcglaciers.org/_images/ESA_logo.svg" width="160" align='right'/>
</div>

# Further DTC-Glaciers features, work in progress

If necessary, install the DTCG API with:

```
!pip install 'dtcg[jupyter] @ git+https://github.com/DTC-Glaciers/dtcg'
```

In a cell below.

In [ ]:
from dtcg.api.external import call
import dtcg.integration.oggm_bindings as dtcg_oggm
import dtcg.interface.plotting as dtcg_plotting
import holoviews as hv
import xarray as xr

In this notebook we showcase other features of DTC-Glacires which potentially can become usefull in the future.

## Streaming Datacubes

DTCG simplifies user interactions and saves on bandwidth costs by streaming data.
Streaming a datacube just requires the name of the glacier:

In [ ]:
streamer = call.StreamDatacube()
datacube = streamer.stream_datacube(glacier="Bruarjökull")
datacube

We can also specify the datacube layer we want to access by using the ``layer`` argument:

In [ ]:
datacube_l1 = streamer.stream_datacube(glacier="Bruarjökull", layer="L1")
datacube_l1

Let's take a look at 2015, [which we know from another notebook was an anomalous year for Iceland](https://notebooks.dtcglaciers.org/notebooks/04_calibrate_with_eolis_data.ipynb).
We can compare the cumulative specific mass balance **observed by CryoSat-2** to the cumulative specific mass balance **modelled by OGGM**.
To make things simpler, we also combine the data with some glacier attributes from OGGM.
Again, we can do this using DTCG with just a few lines of code.

In [ ]:
rgi_id = "RGI60-06.00377"
dtcg_data = dtcg_oggm.BindingsCryotempo()
gdir = dtcg_data.get_glacier_directories(rgi_ids=[rgi_id], from_prepro_level=4, prepro_border=80)[0]
runoff_data = dtcg_data.get_aggregate_runoff(gdir=gdir)

We can plot this interactively.

In [ ]:
graph_artist = dtcg_plotting.BokehGraph()
artist = dtcg_plotting.BokehCryotempo()
fig_smb_cumulative = artist.plot_eolis_smb(
    datacube=datacube.L1, ref_year=2015, glacier_area=gdir.rgi_area_km2
).opts(title="a) Cumulative Specific Mass Balance (CryoSat)")
fig_runoff_cumulative = graph_artist.plot_runoff_timeseries(
    runoff=runoff_data["monthly_runoff"],
    ref_year=2015,
    cumulative=True,
).opts(title="b) Cumulative Monthly Runoff (OGGM)")
hv.Layout([fig_smb_cumulative, fig_runoff_cumulative]).opts(sizing_mode="stretch_width").opts(title="Bruarjökull, Iceland").cols(1)

We can see from **Figure a)** that Bruarjökull experienced a significant delay to the start of the 2015 summer season, which led to the highest winter cumulative specific mass balance since at least 2011.
**Figure b)** shows a corresponding decrease in monthly runoff. Indeed, [Landsvirkjun considered the annual inflow into Vatnajökull to be "very dry"](https://gogn.lv.is/files/Arsskyrslur/LV-Annual_Report_2015.pdf).


### Technical Implementation

Datacubes are streamed directly from a URL, and opened as data trees.
This is equivalent to accessing a ``GeoZarrHandler``'s ``data_tree`` attribute:

```
xr.open_datatree(
    stream_url="https://cluster.klima.uni-bremen.de/~dtcg/test_zarr/",
    group=None,
    chunks={},
    engine="zarr",
    consolidated=True,
    decode_cf=True,
)
```

A user will not download data unless they perform a processing step or call ``compute()``.

In [ ]:
datacube.L1.eolis_elevation_change_sigma_timeseries

## DTC-Glaciers API

In [ ]:
import dtcg
import dtcg.interface.gateway
import dtcg.interface.plotting

A user interacts with DTCG only through API requests.
This means a single centralised framework can handle requests from Jupyter notebooks, websites, and cli wrappers.
Flows are simplified for non-technical users, and can be customised for more advanced use cases.
It also prevents significant changes to the DTCG API interfering with existing flows, as little to no backend code is exposed to the user.

In [ ]:
# User selects these via dropdown menus
subregion_name = "vent_rofenache"
glacier_name = "Hintereisferner"

API queries are extensible, as long as they conform to the OpenAPI standard.
For selecting a subregion, this is what an API query could look like:

In [ ]:
# A query might look like this
user_query_params = {
    "action": "select_subregion",
    "region_name": "Central Europe",
    "subregion_name": subregion_name,
    "glacier_name": glacier_name,
    "shapefile_path": "nested_catchments_oetztal/nested_catchments_oetztal.shx",
    "oggm_params": {
        "use_multiprocessing": True,
        "rgi_version": "62",
        "store_model_geometry": True,
    },
}

A user can also select specific glaciers:

In [ ]:
# A query might look like this
user_query_params = {
    "action": "select_glacier",
    "region_name": "Central Europe",
    "subregion_name": subregion_name,
    "glacier_name": glacier_name,
    "shapefile_path": "nested_catchments_oetztal/nested_catchments_oetztal.shx",
    "oggm_params": {
        "use_multiprocessing": True,
        "rgi_version": "62",
        "store_model_geometry": True,
    },
}
handler = dtcg.interface.gateway.GatewayHandler(query=user_query_params)
assert handler.response["response_code"] == "200"
data = handler.response["data"]

The API is very flexible: it can pass OGGM parameters directly to OGGM, and to preserve bandwidth a response can be customised to contain as little data as needed.

Components are standalone, and can be combined into different dashboards.

For this example, the response data are synthesised into interactive plots.
In a dashboard setting, these plots can also support real-time data.

In [ ]:
dashboard = dtcg.interface.plotting.HoloviewsDashboard()
dashboard.plot_graph.plot_mass_balance(
    observations=data["runoff_data"]["wgms"],
    mass_balance=data["runoff_data"]["mass_balance"],
    name=handler.query.glacier_name,
)

All figures are generated using Holoviews, with Bokeh as a backend. Components can be accessed individually, or implemented into a dashboard. These can then be implemented into a frontend interface using FastAPI and Panel for web, and FastAPI and Celery for Jupyter notebooks.

In [ ]:
dashboard = dtcg.interface.plotting.HoloviewsDashboardL2()
dashboard.plot_runoff_dashboard(
    data=data,
    subregion_name=handler.query.subregion_name,
    glacier_name="",
)